### Run main.py

In [1]:
# %run main.py
from api_calls import fetch_item_from_api, fetch_item_to_df, fetch_daily_cookie, fetch_items
from make_graphs import plot_historic_price, plot_historic_volume, plot_simple_moving_average, plot_exponential_moving_average, plot_weighted_least_squares, plot_bollinger_bands, plot_relative_strength_index
from calculate_metrics import calculate_sma, calculate_ema, calculate_bollinger_bands, calculate_price_percentage_change, calculate_relative_strength_index, calculate_market_cap

dailyCookie = fetch_daily_cookie()
items = fetch_items()

current_item = fetch_item_to_df(items[4], dailyCookie)
non_aggregated_item = fetch_item_from_api(items[4], dailyCookie)

# print(items[4])
# print(current_item.tail())
# print(non_aggregated_item.tail())

START_DATE = current_item.index[0]
END_DATE = current_item.index[-1]



### historic_price
 

In [2]:
import plotly.graph_objects as go
import pandas as pd
from ipywidgets import interact, DatePicker
import numpy as np


df = current_item.copy()


start_date_picker = DatePicker(description='Start Date', value=START_DATE)
end_date_picker = DatePicker(description='End Date', value=END_DATE)


interact(lambda start_date, end_date: plot_historic_price(df, start_date, end_date),
         start_date=start_date_picker, end_date=end_date_picker)

interactive(children=(DatePicker(value=Timestamp('2016-11-29 00:00:00'), description='Start Date', step=1), Da…

<function __main__.<lambda>(start_date, end_date)>

### historic_volume

In [3]:
# > Volume Indicatior (1)

import plotly.graph_objects as go
import pandas as pd
from ipywidgets import interact, DatePicker
import numpy as np

df = current_item.copy()


start_date_picker = DatePicker(description='Start Date', value=START_DATE)
end_date_picker = DatePicker(description='End Date', value=END_DATE)


interact(lambda start_date, end_date: plot_historic_volume(df,  start_date, end_date),
         start_date=start_date_picker, end_date=end_date_picker)

interactive(children=(DatePicker(value=Timestamp('2016-11-29 00:00:00'), description='Start Date', step=1), Da…

<function __main__.<lambda>(start_date, end_date)>

### simple_moving_average

In [4]:
# SIMPLE Moving Average (SMA)                            


import plotly.graph_objects as go
import pandas as pd
from ipywidgets import interact, DatePicker
import numpy as np


df = current_item.copy()
# NOTE: Window = sets the period for calculating the moving average.
window = 2
df = calculate_sma(df, window)


start_date_picker = DatePicker(description='Start Date', value=START_DATE)
end_date_picker = DatePicker(description='End Date', value=END_DATE)


interact(lambda start_date, end_date:
            plot_simple_moving_average(df, start_date, end_date),
            start_date=start_date_picker, end_date=end_date_picker)



interactive(children=(DatePicker(value=Timestamp('2016-11-29 00:00:00'), description='Start Date', step=1), Da…

<function __main__.<lambda>(start_date, end_date)>

### exponential_moving_average

In [5]:
 # Exponential Moving Average (EMA)  
 # | Part of Moving Averages (2) block

import plotly.graph_objects as go
import pandas as pd
from ipywidgets import interact, DatePicker
import numpy as np


df = current_item.copy()

# NOTE: Span = Used to calculate the decay factor. Higher span -> greater exponential smoothing
span = 10
df = calculate_ema(df, span)

start_date_picker = DatePicker(description='Start Date', value=START_DATE)
end_date_picker = DatePicker(description='End Date', value=END_DATE)


interact(lambda start_date, end_date:
            plot_exponential_moving_average(df, start_date, end_date),
            start_date=start_date_picker, end_date=end_date_picker)

interactive(children=(DatePicker(value=Timestamp('2016-11-29 00:00:00'), description='Start Date', step=1), Da…

<function __main__.<lambda>(start_date, end_date)>

### ordinary_least_squares & weighted_least_squares (ML)

In [6]:
# Weighted Least Squares Estimator (3.2) - 
# Note: this makes OLS (above) redundant since you can adjust weighting to 1 in the 'weight' array

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
import plotly.graph_objects as go
import pandas as pd
from ipywidgets import interact, DatePicker
import numpy as np


df = current_item.copy()

# Weights = determine the influence of data points in WLS; using weights of 1,1 yields OLS results (OLS has uniform weightings).

df['date_ordinal'] = df.index.map(pd.Timestamp.toordinal)
X = df['date_ordinal'].values.reshape(-1, 1)
y = df['price_usd'].values

weights = np.linspace(1, 10, len(X))

model_wls = sm.WLS(y, sm.add_constant(X), weights=weights).fit()
y_pred = model_wls.predict(sm.add_constant(X))

split_point = int(len(X) * 0.80)
X_train, X_test = X[:split_point], X[split_point:]
y_train, y_test = y[:split_point], y[split_point:]





start_date_picker = DatePicker(description='Start Date', value=df.index[0], disabled=False)
end_date_picker = DatePicker(description='End Date', value=df.index[-1], disabled=False)


interact(lambda start_date, end_date:
            plot_weighted_least_squares(df, start_date, end_date, y_pred, split_point, y_test),
            start_date=start_date_picker, end_date=end_date_picker)





interactive(children=(DatePicker(value=Timestamp('2016-11-29 00:00:00'), description='Start Date', step=1), Da…

<function __main__.<lambda>(start_date, end_date)>

### bollinger_bands

In [7]:
# # Bollinger Bands - Bolligner Bands currently use the SMA for the upper + lower band, but standardised to historical price                           


import plotly.graph_objects as go
import pandas as pd
from ipywidgets import interact, DatePicker
import numpy as np


df = current_item.copy()

# Window sets the period length for calculating the Bollinger Bands moving average. Industry standard is 20d. 
window = 20
df= calculate_bollinger_bands(df, window)


start_date_picker = DatePicker(description='Start Date', value=START_DATE)
end_date_picker = DatePicker(description='End Date', value=END_DATE)

interact(lambda start_date, end_date:
            plot_bollinger_bands(df, start_date, end_date),
            start_date=start_date_picker, end_date=end_date_picker)

interactive(children=(DatePicker(value=Timestamp('2016-11-29 00:00:00'), description='Start Date', step=1), Da…

<function __main__.<lambda>(start_date, end_date)>

### price_percentage_change

In [8]:
# Percentage Change output

from ipywidgets import DatePicker, interact

df = current_item.copy()

start_date_picker = DatePicker(description='Start Date', value=START_DATE)
end_date_picker = DatePicker(description='End Date', value=END_DATE)


def update_percentage_change(start_date, end_date):
    calculate_price_percentage_change(df, start_date, end_date)

interact(update_percentage_change, start_date=start_date_picker, end_date=end_date_picker)


interactive(children=(DatePicker(value=Timestamp('2016-11-29 00:00:00'), description='Start Date', step=1), Da…

<function __main__.update_percentage_change(start_date, end_date)>

### Relative_strength_index

In [9]:
#rsi 

import plotly.graph_objects as go
import pandas as pd
from ipywidgets import interact, DatePicker
import numpy as np

 
df = non_aggregated_item.copy()

window = "5d"
rsi_data = calculate_relative_strength_index(df, window)


start_date_picker = DatePicker(description='Start Date', value=START_DATE)
end_date_picker = DatePicker(description='End Date', value=END_DATE)

interact(lambda start_date, end_date:
            plot_relative_strength_index(start_date, end_date, rsi_data),
            start_date=start_date_picker, end_date=end_date_picker)



    

interactive(children=(DatePicker(value=Timestamp('2016-11-29 00:00:00'), description='Start Date', step=1), Da…

<function __main__.<lambda>(start_date, end_date)>

### CANDLESTICK - NOT DONE YET - PLAN: IMPLIMENT IN EVERY CODEBLOCK AS TRUE/FALSE FUNCTION

In [10]:
#CANDLESTICK BUT WITH FIRST AND LAST INSTEAD OF MEAN AND MEDIAN

import plotly.graph_objects as go
import pandas as pd
from ipywidgets import interact, DatePicker
import numpy as np

# renames current item to df
df = current_item.copy()

# Resample the data to weekly timeframe
# NOTE: Range can be altered: 'W' (week), 'M' (Month), '2D' (2 Days)
new_data = df.resample('w').agg({
    'price_usd': ['max', 'min', 'first', 'last'],
    'volume': 'sum'
})


# Slices the multi-level columns and renames
new_data.columns = ['high', 'low', 'open', 'close' ,'volume']


# Function to update the graph based on the selected date range
def update_graph(start_date, end_date):
 

    mask = (new_data.index >= pd.to_datetime(start_date)) & (new_data.index <= pd.to_datetime(end_date))
    filtered_data = new_data.loc[mask]

    
    # Creating the candlestick chart
    fig = go.Figure(data=[go.Candlestick(x=filtered_data.index,
                                         open=filtered_data['open'],
                                         high=filtered_data['high'],
                                         low=filtered_data['low'],
                                         close=filtered_data['close'],
                                         name='Candlestick')])
    


    # Formatting
    fig.update_layout(
        title='SMA - Candlestick',
        xaxis_title='Date',
        yaxis_title='Price USD',
        yaxis=dict(
            title='Price USD',
            titlefont_size=16,
            tickfont_size=14,
        ),
        
        showlegend=True
    )

    fig.show()




# Creating interactive widgets for date selection
start_date_picker = DatePicker(description='Start Date', value=START_DATE)
end_date_picker = DatePicker(description='End Date', value=END_DATE)

# Display the interactive widget

interact(update_graph, start_date=start_date_picker, end_date=end_date_picker)

interactive(children=(DatePicker(value=Timestamp('2016-11-29 00:00:00'), description='Start Date', step=1), Da…

<function __main__.update_graph(start_date, end_date)>

Market Cap

In [11]:
# Market Cap Calculation

df = non_aggregated_item.copy()
market_cap = df.resample('d').agg({
 
    'price_usd': ['last'],
    'volume': 'sum'
      })
     # Slices the multi-level columns and renames
market_cap.columns = ['close','volume']




def update_market_cap(start_date, end_date):
     calculate_market_cap(market_cap, start_date, end_date)

# Create interactive widgets
start_date_picker = DatePicker(description='Start Date', value=START_DATE)
end_date_picker = DatePicker(description='End Date', value=END_DATE)

interact(update_market_cap, start_date=start_date_picker, end_date=end_date_picker)



interactive(children=(DatePicker(value=Timestamp('2016-11-29 00:00:00'), description='Start Date', step=1), Da…

<function __main__.update_market_cap(start_date, end_date)>